In [1]:
# all the utility files,code written in python 2.7

In [1]:
import sys
import os
import csv
import numpy as np
import scipy.io.wavfile


class Instance(object):
    """
        Instance class represents set of raw data collected per data instance
    """
    def __init__(self, dir):
        self.audio = self._load_audio(dir)
        self.touch = self._load_touch_data(dir)
        self.info = self._load_instance_info(dir)

    def _load_audio(self, dir):
        """ load audio data
            param dir: a path to an instance directory
            return: audio data
        """
        rate, wav = scipy.io.wavfile.read(os.path.join(dir, "audio.wav"))
        return wav
        
    def _load_touch_data(self, dir):
        """ load touch data
            param dir: a path to an instance directory
            return : a dictionary contains touch data
        """
        with open(os.path.join(dir, "touch.csv"), "rU") as f:
            reader = csv.DictReader(f)
            for touch in reader:
                for key in  touch.keys():
                    touch[key] = float(touch[key])
                break
        return touch
    
    def _load_instance_info(self, dir):
        """ load instance info from a directory path
            param dir: a path to an instance directory
            return: a dictionary contains basic instance information
        """
        info = {}
        user_dirnames = os.path.basename(os.path.dirname(dir)).split("-")
        info["surface"] = user_dirnames[0]
        info["user"] = user_dirnames[1]
        instance_dirnames = os.path.basename(dir).split("-")
        info["timestamp"] = instance_dirnames[0]
        # set None to classlabel if it's test data
        info["classlabel"] = instance_dirnames[1] if len(instance_dirnames) == 2 else None
        return info
def load_instances(dir):
    #print dir
    """ function for loading raw data instances
        param dir: a path to a data directory (i.e. task_data/train or task_data/test)
        return: a list of data instance objects
    """
    instances = []
    #print instances
    for root, dirs, files in os.walk(os.path.join(dir)):
        #print root,dirs,files
        for filename in files:
            if filename == "audio.wav":
                instances.append(Instance(root))
    return instances

def load_labels(instances):
    """ load class labels
        param instances: a list of data instance objects
        return: class labels mapped to a number (0=pad, 1=knuckle)
    """
    y = np.array([{"pad": 0, "knuckle": 1}[instance.info["classlabel"]] for instance in instances], dtype=int)
    return y


def load_timestamps(instances):
    """ load timestamps
        param instances: a list of data instance objects
    """
    timestamps = [instance.info["timestamp"] for instance in instances]
    return timestamps


def convert_to_classlabels(y):
    """ convert to classlabels
        param y: mapped class labels
        return: class labels
    """
    classlabels = [["pad", "knuckle"][y[i]] for i in range(len(y))]
    return classlabels


def write_results(timestamps, classlabels, output):
    """ write classification results to an output file
        param timestamps: a list of timestamps
        param classlabels: a list of predicted class labels
        return : None
    """
    if len(timestamps) != len(classlabels):
        raise Exception("The number of timestamps and classlabels doesn't match.")
    with open(output, "wb") as f:
        f.write("timestamp,label\n")
        for timestamp, classlabel in zip(timestamps, classlabels):
            f.write(timestamp + "," + classlabel + "\n")

In [2]:
# step 1 loading all the data content using the utility files above
train_instances = load_instances('./data/train')

In [3]:
# loading the audio files, using scikit talkbox library to extract MFCC from those loaded files, library can 
#installed using setup.py in scikit talkbox folder attached , using command "python setup.py install" if any problem
# there is a readme.txt explaining more in detail
X = np.array([instance.audio.astype(float) for instance in train_instances])# audio files
print X, X.shape
from scikits.talkbox.features import mfcc
p=np.zeros(shape=(20659L, 13))
for t,y in enumerate(X):
    ceps, mspec, spec = mfcc(y)
    p[t]=ceps
print p.shape ,p # p containg 13 mfcc coefficients for all the samples

[[-1464. -1485. -1474. ...,  -898.  -898.  -896.]
 [-1219. -1197. -1214. ...,  -934.  -925.  -938.]
 [-1273. -1288. -1270. ...,  -735.  -767.  -772.]
 ..., 
 [-2014. -2003. -1980. ..., -1847. -1907. -1944.]
 [-2015. -2008. -1996. ..., -1995. -1996. -1982.]
 [-2000. -2008. -1997. ..., -2091. -2092. -2068.]] (20659L, 256L)
(20659L, 13L) [[  8.9284845    1.27909013  -0.16942932 ...,   0.07501468   0.20557355
   -0.08660229]
 [  5.45871065  -0.25808326   0.25652745 ...,  -0.12637589   0.05453893
   -0.0379106 ]
 [  7.34200833   0.43895332   0.63722168 ...,  -0.12781804  -0.04497108
   -0.04989088]
 ..., 
 [ 11.2525355    2.27716453  -0.85784616 ...,  -0.09037062  -0.05465509
   -0.07346418]
 [ 11.30302956   2.29938476  -1.1319388  ...,  -0.10776011   0.35115705
   -0.03699211]
 [ 10.24428125   2.19161288  -0.7221009  ...,  -0.11672084   0.22781351
    0.07169545]]


In [4]:
#loading touch information and extracting out all important features (including major and minor axis of ellipse)
X1=np.array([instance.touch for instance in train_instances])
print X1
dictlist=[]
l=np.zeros(shape=(20659L, 6))
for i in range(X1.shape[0]):
    for key, value in X1[i].iteritems():
        temp = value
        dictlist.append(temp)
    l[i]=dictlist
    dictlist=[]
print l.shape # l consists of all the 6 features 
feat=np.concatenate([p, np.atleast_2d(l[:,0]).T, np.atleast_2d(l[:,5]).T],axis=1) # all the features selected from audio to touch
#information 15 (13 MFCC and 2 major and minor axis )
print feat.shape,feat

[ {'major': 6.0, 'orientation': -1.0, 'pressure': 0.0, 'y': 1647.0, 'x': 176.0, 'minor': 6.0}
 {'major': 5.0, 'orientation': -1.0, 'pressure': 0.0, 'y': 1666.0, 'x': 152.0, 'minor': 5.0}
 {'major': 5.0, 'orientation': -1.0, 'pressure': 0.0, 'y': 1703.0, 'x': 410.0, 'minor': 5.0}
 ...,
 {'major': 5.0, 'orientation': -1.0, 'pressure': 0.0, 'y': 1784.0, 'x': 707.0, 'minor': 4.0}
 {'major': 4.0, 'orientation': -1.0, 'pressure': 0.0, 'y': 1754.0, 'x': 906.0, 'minor': 4.0}
 {'major': 4.0, 'orientation': -1.0, 'pressure': 0.0, 'y': 1778.0, 'x': 952.0, 'minor': 4.0}]
(20659L, 6L)
(20659L, 15L) [[  8.9284845    1.27909013  -0.16942932 ...,  -0.08660229   6.           6.        ]
 [  5.45871065  -0.25808326   0.25652745 ...,  -0.0379106    5.           5.        ]
 [  7.34200833   0.43895332   0.63722168 ...,  -0.04989088   5.           5.        ]
 ..., 
 [ 11.2525355    2.27716453  -0.85784616 ...,  -0.07346418   5.           4.        ]
 [ 11.30302956   2.29938476  -1.1319388  ...,  -0.036992

In [5]:
# load labels for the training data
y_train = load_labels(train_instances)# labels for the respective samples
print y_train
x_temp=np.concatenate([feat, np.atleast_2d(y_train).T],axis=1)
print x_temp.shape,x_temp
np.random.shuffle(x_temp) #randomly shufle data before processing for better estimation of predicted results
feat=x_temp[:,0:15] ## data with 15 features including major and minor axis
feattt=x_temp[:,0:13] # data with 13 features excluding them
y_train=x_temp[:,15]
print feat.shape,y_train.shape,y_train,feat,feattt.shape

[0 0 0 ..., 0 0 0]
(20659L, 16L) [[  8.9284845    1.27909013  -0.16942932 ...,   6.           6.           0.        ]
 [  5.45871065  -0.25808326   0.25652745 ...,   5.           5.           0.        ]
 [  7.34200833   0.43895332   0.63722168 ...,   5.           5.           0.        ]
 ..., 
 [ 11.2525355    2.27716453  -0.85784616 ...,   5.           4.           0.        ]
 [ 11.30302956   2.29938476  -1.1319388  ...,   4.           4.           0.        ]
 [ 10.24428125   2.19161288  -0.7221009  ...,   4.           4.           0.        ]]
(20659L, 15L) (20659L,) [ 0.  0.  1. ...,  0.  0.  1.] [[  7.86932862e+00   1.22608086e+00   2.44270648e-01 ...,   1.76541707e-01
    5.00000000e+00   5.00000000e+00]
 [  5.48207805e+00  -2.93169084e-01   5.04493140e-01 ...,  -5.11743783e-02
    5.00000000e+00   5.00000000e+00]
 [  1.29950740e+01   2.72297567e+00  -2.04334512e+00 ...,  -2.41050454e-01
    4.00000000e+00   4.00000000e+00]
 ..., 
 [  6.55007260e+00   8.54746678e-01   5.40850

In [6]:
# model selection 

In [8]:
# Scikit learn library used to import all the models
from sklearn.svm import SVR
from sklearn import svm
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import cross_validation
from sklearn.feature_selection import RFE
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import cross_validation
from sklearn.feature_selection import RFE
from sklearn.ensemble import AdaBoostClassifier
clf10 = AdaBoostClassifier(n_estimators=50)
log=LogisticRegression(penalty='l1',C=1)
decision = DecisionTreeClassifier( min_samples_split=3)
svc = svm.SVC(kernel='linear', C=1)
svcrbf = svm.SVC(kernel='rbf', C=1)
neigh = KNeighborsClassifier(n_neighbors=3)
gnb = GaussianNB()

In [9]:
# differnt models tried with default parameters with feature engineered dataset
scores1234=cross_validation.cross_val_score(svc, feattt, y_train, cv=5)
scores1234q=cross_validation.cross_val_score(log, feattt, y_train, cv=5)
scores1234r=cross_validation.cross_val_score(decision, feattt, y_train, cv=5)
scores1234s=cross_validation.cross_val_score(neigh,feattt, y_train, cv=5)
scores1234t=cross_validation.cross_val_score(gnb,feattt, y_train, cv=5)
scores1234u=cross_validation.cross_val_score(svcrbf,feattt, y_train, cv=5)
scores1234s1=cross_validation.cross_val_score(clf10,feattt, y_train, cv=5)
print scores1234.mean(),scores1234q.mean(),scores1234r.mean(),scores1234s.mean(),scores1234t.mean(),scores1234u.mean(),scores1234s1.mean()

0.897816968624 0.897429676638 0.891911158354 0.93184553876 0.885086411317 0.922164399086 0.900479105988


In [11]:
# bar plot
import matplotlib.pyplot as plt
values =[scores1234.mean(),scores1234q.mean(),scores1234r.mean(),scores1234s.mean(),scores1234t.mean(),scores1234u.mean(),scores1234s1.mean()]
inds   =np.arange(7)
labels = ["Linear SVM","LOGISTIC REG","Decision Trees","KNN","Naive Bayes","RBFSVM","Adaboost(DT)"]

#Plot a bar chart
plt.figure(1, figsize=(6,4))  #6x4 is the aspect ratio for the plot
plt.bar(inds, values, align='center') #This plots the data
plt.grid(True) #Turn the grid on
plt.ylabel("Accuracy") #Y-axis label
plt.xlabel("Method") #X-axis label
plt.title("QEEXO dataset") #Plot title
plt.xlim(-0.5,2.5) #set x axis range
plt.ylim(0,1) #Set yaxis range

#Set the bar labels
plt.gca().set_xticks(inds) #label locations
plt.gca().set_xticklabels(labels) #label values

#Make sure labels and titles are inside plot area
plt.tight_layout()

#Save the chart
#plt.savefig("../Figures/digits1.pdf")
plt.show()

In [15]:
# applying cross validation to find optimal hyper parameter C for schosen model SVM RBF kernel
K=[3,5,7,9,11]
fhg=[]
for i in K:
    neight = KNeighborsClassifier(n_neighbors=i)
    scores1234u=cross_validation.cross_val_score(neight,feattt, y_train, cv=5)
    print scores1234u.mean()
    fhg.append(scores1234u.mean())

plt.figure(1, figsize=(6,4))  #6x4 is the aspect ratio for the plot
plt.plot(K,fhg,'or-', linewidth=3) #Plot the first series in red with circle marker#Plot the first series in blue with square marker

#This plots the data
plt.grid(True) #Turn the grid on
plt.ylabel("accuracy") #Y-axis label
plt.xlabel("K values") #X-axis label
plt.title("K vs accuracy Values") #Plot title
plt.xlim(-0.1,13) #set x axis range
plt.ylim(0.7,1) #Set yaxis range

#Make sure labels and titles are inside plot area
plt.tight_layout()

#Save the chart
#plt.savefig("../Figures/linegraph1.pdf")
plt.show()


0.93184553876
0.934653036576
0.934556278023
0.934459355432
0.932765319147


In [9]:
# chosen model with optimal hyper parameter is fitted to the training data and predicted on test data ,values are stored in a 
#csv file
test_instances = load_instances("./data/test")
X_test = np.array([instance.audio.astype(float) for instance in test_instances])# audio files
print X_test, X_test.shape
from scikits.talkbox.features import mfcc
p_test=np.zeros(shape=(10528L, 13))
for t,y in enumerate(X_test):
    ceps, mspec, spec = mfcc(y)
    p_test[t]=ceps
print p_test.shape ,p_test 
feat_test=p_test # all the features selected from audio 13 MFCC 
print feat_test.shape
neight = KNeighborsClassifier(n_neighbors=5)
scores1234u=cross_validation.cross_val_score(neight,feattt, y_train, cv=5)
print scores1234u.mean()
neight.fit(feattt,y_train)
y_test=neight.predict(feat_test)
y_test=y_test.astype(int)
timestamps = load_timestamps(test_instances)
classlabels = convert_to_classlabels(y_test)
write_results(timestamps, classlabels, "./fingersense-test-labels.csv")

[[-1627. -1646. -1637. ..., -1530. -1561. -1547.]
 [-1579. -1564. -1584. ..., -1871. -1873. -1866.]
 [-1751. -1747. -1729. ..., -2153. -2154. -2141.]
 ..., 
 [-2047. -2049. -2047. ..., -2111. -2016. -2000.]
 [-2047. -2051. -2098. ..., -2175. -2143. -2121.]
 [-2062. -2040. -2048. ..., -2345. -2280. -2264.]] (10528L, 256L)
(10528L, 13L) [[  8.35596137e+00   2.43506263e+00   1.28172096e+00 ...,  -1.53605112e-01
    8.73525735e-02  -8.81050329e-02]
 [  1.02174225e+01   3.13441687e+00   1.55497149e+00 ...,  -5.19709272e-01
    2.62648674e-01  -3.13623564e-02]
 [  8.96760568e+00   2.28227244e+00   1.60344381e+00 ...,  -1.26103883e-01
   -2.39157673e-01   1.65562771e-01]
 ..., 
 [  1.65464938e+01   1.91863599e+00  -2.68081348e+00 ...,   2.05709841e-01
    8.54158801e-02  -6.76919755e-02]
 [  1.58946592e+01   1.76845684e+00  -2.04156575e+00 ...,  -1.22442740e-01
    1.59359787e-01  -2.97846607e-01]
 [  7.00045578e+00   1.21046737e+00   1.20473140e+00 ...,  -1.13147275e-02
   -7.38034810e-02  -

In [22]:
#Other different features tried to use using python_speech_features library , installed exactly same as talkbox using command  
# python setup.py install, turned out these featurs didnt make much changes, so they were not considered later
from python_speech_features import delta
p2=np.zeros(shape=(20659,13))
for t,y in enumerate(feattt):
    delt=delta(y,1)
    p2[t]=delt
print p2.shape,p2
feat3=np.concatenate([feattt,p2],axis=1)
print feat3.shape
scores1234s1=cross_validation.cross_val_score(svcrbf,feat3, y_train, cv=5)
print scores1234s1.mean()

(20659L, 13L) [[ -5.03647090e+00  -7.01778431e+00  -1.61908719e+00 ...,   3.25365885e-01
    1.18287542e-01   5.26893813e-02]
 [ -5.71450571e+00  -7.65847435e+00  -6.29332866e-01 ...,   7.31211407e-03
   -6.41196483e-03   2.72746185e-02]
 [ -7.11176578e+00  -7.62313820e+00   1.50332206e+00 ...,   1.64155032e-02
    1.08035092e-01  -1.61774716e-01]
 ..., 
 [ -2.81847595e+00  -2.58776997e+00   3.51470334e-02 ...,  -2.02372380e-02
    1.96083877e-01   1.66761052e-01]
 [ -2.89219574e+00  -2.74333269e+00   1.66031734e-01 ...,   1.35663867e-02
    1.27943875e-02  -3.29139905e-02]
 [ -3.48581124e+00  -3.85048823e+00  -2.14277174e-01 ...,  -6.05145976e-03
    9.07353629e-02  -6.65807550e-02]]
(20659L, 26L)
0.919066613894


In [23]:
# out of training data some data was kept aside to test model accuracy as we know the labels of them, turns out it wrongly labels 8out 
# 244
testtt=feattt[18256:18500,:]
y_Testt=y_train[18256:18500]
neight = KNeighborsClassifier(n_neighbors=5)
ghj=np.concatenate([feattt[0:18256,:], feattt[18500:,:]])
lkj=np.concatenate([y_train[0:18256], y_train[18500::]])
print ghj.shape,ghj, lkj.shape,lkj
neight.fit(ghj,lkj)
ypredd=neight.predict(testtt)
print ypredd.shape
print ypredd,y_Testt,ypredd==y_Testt

(20415L, 13L) [[  1.31489550e+01   3.07601325e+00  -8.86613587e-01 ...,   1.54296890e-01
    2.85493213e-01   3.90871975e-01]
 [  1.30391795e+01   1.61016807e+00  -2.27776922e+00 ...,   3.01272852e-02
   -3.72458813e-02   1.73033556e-02]
 [  1.28017153e+01  -1.42181626e+00  -2.44456110e+00 ...,  -3.44790274e-01
    1.94829341e-01  -1.28720090e-01]
 ..., 
 [  6.55621939e+00   9.19267501e-01   1.38067946e+00 ...,  -1.81457387e-01
   -1.22811737e-01   2.10710367e-01]
 [  5.55614122e+00  -2.28250274e-01   6.94758372e-02 ...,   2.39675803e-03
    9.38135140e-02   2.79855330e-02]
 [  8.50292876e+00   1.53130628e+00   8.01952297e-01 ...,  -4.77362685e-02
    2.66895967e-01   1.33734457e-01]] (20415L,) [ 1.  1.  1. ...,  0.  0.  0.]
(244L,)
[ 0.  0.  1.  0.  0.  1.  0.  0.  1.  0.  0.  0.  1.  0.  0.  1.  1.  0.
  1.  0.  0.  0.  1.  0.  0.  1.  0.  1.  0.  1.  1.  0.  1.  0.  0.  1.
  0.  0.  1.  0.  0.  0.  0.  0.  1.  1.  0.  0.  1.  1.  1.  0.  1.  1.
  0.  1.  1.  0.  0.  0.  1.  1.  1.  

In [24]:
#try to increase the features size using basis expansion technique turns out it didnt increase accuracy as well.
from sklearn.preprocessing import PolynomialFeatures
featpoly= PolynomialFeatures(3)
featpol=featpoly.fit_transform(feattt)
print feattt.shape,featpol.shape
neight = KNeighborsClassifier(n_neighbors=7)
scores1234u=cross_validation.cross_val_score(neight,feattt, y_train, cv=5)
print scores1234u.mean()

(20659L, 13L) (20659L, 560L)
0.934556278023
